In [ ]:
# Ground truth
# Azure AI search -> retrieve the chunk based on questions
# Preprocessing for SLM and LLM (another SLM part of inference)
# Context (list of chunks) and question to inference script (It will be handled internally in scoring script)
# Convert the context and question into appropriate format
# LLM -> response
# SLM -> response
# Evaluation metrics ( LLM response, LLM context, SLM response , SLM context and ground truth) -> 100 QA pair
# RAGAS
# Answer response time.
# Totel token count
# Box plot of each metrics



In [194]:
# Read ground truth
import pandas as pd
import json

# Path to your JSONL file
file_path = "lab_maintenance_100_qa.jsonl"

# Read the JSON file as a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)  # Use json.load() since it's a JSON array, not a JSONL file

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()


,question,answer,chapter
0,What is the main purpose of the microplate rea...,The microplate reader is used to read the resu...,1
1,What type of test is a microplate reader prima...,It is primarily used for the ELISA (Enzyme-Lin...,1
2,Describe the wavelength range typically used b...,Microplate readers typically operate within a ...,1
3,What are the key components required for an EL...,"Key components include a microplate reader, mi...",1
4,What are the different phases involved in an E...,ELISA involves coating wells with antibodies/a...,1


In [180]:
# import json
# import urllib.request
# import os
# import ssl

# def allowSelfSignedHttps(allowed):
#     """Bypass the server certificate verification on the client side."""
#     if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
#         ssl._create_default_https_context = ssl._create_unverified_context

# allowSelfSignedHttps(True)  # This is needed if you use self-signed certificates.

# def send_chat_input(chat_input):
#     """Send a chat input to the API and return the parsed response."""
#     # API request payload
#     data = {
#         "chat_input": chat_input,
#         "chat_history": []  # Modify this if you have previous chat history
#     }
    
#     # API credentials
#     api_key = "OzYEqfLg8ueVePHKun3UejfBMLphos4N"  # Replace with your actual API key
#     url = "https://ws-meemankraft-gcmko.eastus2.inference.ml.azure.com/score"  # Replace with the actual API URL

#     # Prepare request body and headers
#     body = str.encode(json.dumps(data))
    
#     if not api_key:
#         raise Exception("A valid API key is required to invoke the endpoint.")
    
#     headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + api_key}
#     req = urllib.request.Request(url, body, headers)
    
#     # Initialize response structure
#     text_docs = []
#     response_dict = {}

#     try:
#         # Make the API call
#         response = urllib.request.urlopen(req)
#         result = response.read().decode('utf-8')
        
#         # Parse the JSON response
#         result_dict = json.loads(result)
#         chat_output = result_dict.get('chat_output', {})
        
#         # Extract chat response data
#         response_output = chat_output.get('response', 'No response found')
#         user_question = chat_output.get('user_question', 'No question found')
#         source_documents = chat_output.get('source_documents', [])
        
#         # Collect source document texts
#         for doc in source_documents:
#             text = doc.get('text', 'No text found')
#             text_docs.append(text)
        
#         # Build the response dictionary
#         response_dict['response_output'] = response_output
#         response_dict['user_question'] = user_question
#         response_dict['source_documents'] = text_docs

#         return response_dict

#     except urllib.error.HTTPError as error:
#         print(f"The request failed with status code: {error.code}")
#         print(error.info())
#         print(error.read().decode("utf8", 'ignore'))
#         return None

# # Example usage: Call the function with different chat inputs
# z = send_chat_input("What is the main purpose of the microplate reader?")
# print(z)


In [158]:
df = df.head(3)
df

,question,answer,chapter
0,What is the main purpose of the microplate rea...,The microplate reader is used to read the resu...,1
1,What type of test is a microplate reader prima...,It is primarily used for the ELISA (Enzyme-Lin...,1
2,Describe the wavelength range typically used b...,Microplate readers typically operate within a ...,1


In [181]:
# # Apply the function to each row in the DataFrame
# df['LLM_response'] = df['question'].apply(lambda q: send_chat_input(q))

# # If you want to separate the response_output and source_documents into different columns
# df['response_output'] = df['LLM_response'].apply(lambda x: x['response_output'])
# df['source_documents'] = df['LLM_response'].apply(lambda x: x['source_documents'])

# # Drop the intermediate column if no longer needed
# # df = df.drop(columns=['LLM_response'])

In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score

nltk.download('punkt')

# Function to calculate all metrics
def calculate_metrics(ground_truth, llm_response):
    # ROUGE score calculation
    def calculate_rouge(reference, hypothesis):
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(reference, hypothesis)
        return scores

    # BERTScore calculation
    def calculate_bertscore(reference, hypothesis):
        P, R, F1 = score([hypothesis], [reference], lang="en", verbose=False)
        return {
            'precision': P.item(),
            'recall': R.item(),
            'f1': F1.item()
        }

    # Calculating BLEU
    # bleu = calculate_bleu(ground_truth, llm_response)
    
    # Calculating ROUGE
    rouge_scores = calculate_rouge(ground_truth, llm_response)
    
    # Calculating BERTScore
    bertscore = calculate_bertscore(ground_truth, llm_response)
    
    # Combine all results
    results = {
        # 'BLEU': bleu,
        'ROUGE-1': rouge_scores['rouge1'].fmeasure,
        'ROUGE-2': rouge_scores['rouge2'].fmeasure,
        'ROUGE-L': rouge_scores['rougeL'].fmeasure,
        'BERTScore Precision': bertscore['precision'],
        'BERTScore Recall': bertscore['recall'],
        'BERTScore F1': bertscore['f1']
    }
    
    return results

df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'BERTScore Precision', 'BERTScore Recall', 'BERTScore F1']] = df.apply(
    lambda row: pd.Series(calculate_metrics(row['answer'], row['LLM_response'])),
    axis=1
)

In [163]:
import os
import time
import urllib.request
import json
import ssl
from deepeval.metrics import AnswerRelevancyMetric, ContextualRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric
from deepeval.test_case import LLMTestCase
from deepeval.models.base_model import DeepEvalBaseLLM
from langchain_openai import AzureChatOpenAI

class AzureOpenAIEvaluator(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"
    

class RAG_Evaluator():
    def __init__(self, framework):
        self.framework = framework
        if self.framework=="deepeval":
            custom_model = AzureChatOpenAI(
                # openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                # azure_deployment=os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT"),
                # azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                # openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                openai_api_version="2023-03-15-preview",
                azure_deployment="gpt4omini",
                azure_endpoint="https://aoi-meemank.openai.azure.com",
                openai_api_key="772624b8d13c4d19a12e77108401695e",
            )
            azure_openai = AzureOpenAIEvaluator(model=custom_model)
            self.answer_relevancy_metric = AnswerRelevancyMetric(
                model=azure_openai,
                threshold=0.7,
                include_reason=True
            )
            self.context_relevancy_metric = ContextualRelevancyMetric(
                model=azure_openai,
                threshold=0.7,
                include_reason=True
            )
            self.context_precision_metric = ContextualPrecisionMetric(
                threshold=0.7,
                model=azure_openai,
                include_reason=True
            )
            self.context_recall_metric = ContextualRecallMetric(
                threshold=0.7,
                model=azure_openai,
                include_reason=True
            )

    
    def check_answer_relevancy(self, question, actual_output):
        score = ""
        reason = ""

        if self.framework=="deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output
            )
            self.answer_relevancy_metric.measure(test_case)
            score = self.answer_relevancy_metric.score
            reason = self.answer_relevancy_metric.reason

        return score, reason


    def check_contextual_relevancy(self, question, actual_output, retrieval_contexts):
        score = ""
        reason = ""
    
        if self.framework=="deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output,
                retrieval_context=retrieval_contexts
            )
            self.context_relevancy_metric.measure(test_case)
            score = self.context_relevancy_metric.score
            reason = self.context_relevancy_metric.reason

        return score, reason
    

    def check_contextual_precision(self, question, actual_output, expected_output, retrieval_contexts):
        score = ""
        reason = ""
    
        if self.framework=="deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output,
                expected_output=expected_output,
                retrieval_context=retrieval_contexts
            )
            self.context_precision_metric.measure(test_case)
            score = self.context_precision_metric.score
            reason = self.context_precision_metric.reason

        return score, reason
    

    def check_contextual_recall(self, question, actual_output, expected_output, retrieval_contexts):
        score = ""
        reason = ""
    
        if self.framework=="deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output,
                expected_output=expected_output,
                retrieval_context=retrieval_contexts
            )
            self.context_recall_metric.measure(test_case)
            score = self.context_recall_metric.score
            reason = self.context_recall_metric.reason

        return score, reason


    def get_evaluation(self, question, expected_response, rag_response):
        rag_response_str = rag_response["response_output"]
        metrics = {}
        answer_relevancy_score, answer_relevancy_reason = self.check_answer_relevancy(question, rag_response_str)
        
        time.sleep(5)

        relevant_contexts = []
        for source_doc in rag_response["source_documents"]:
            relevant_contexts.append(source_doc)
        print("relevant_contexts",relevant_contexts)
        # relevant_contexts=rag_response["source_documents"]
        contextual_relevancy_score, contextual_relevancy_reason = self.check_contextual_relevancy(question, rag_response_str, relevant_contexts)
        time.sleep(5)
        if expected_response != "":
            contextual_precision_score, contextual_precision_reason = self.check_contextual_precision(question, rag_response_str, expected_response, relevant_contexts)
            time.sleep(5)
            contextual_recall_score, contextual_recall_reason = self.check_contextual_recall(question, rag_response_str, expected_response, relevant_contexts)
            time.sleep(5)
        else:
            contextual_precision_score, contextual_precision_reason = 0.0, "NA"
            contextual_recall_score, contextual_recall_reason = 0.0, "NA"
         
        metrics['answer_relevancy_score'] = answer_relevancy_score
        metrics['answer_relevancy_reason'] = answer_relevancy_reason
        metrics['contextual_relevancy_score'] = contextual_relevancy_score
        metrics['contextual_relevancy_reason'] = contextual_relevancy_reason
        metrics['contextual_precision_score'] = contextual_precision_score
        metrics['contextual_precision_reason'] = contextual_precision_reason
        metrics['contextual_recall_score'] = contextual_recall_score
        metrics['contextual_recall_reason'] = contextual_recall_reason
        return metrics

    def get_rag_response(self, question):
        response = ""
        final_dict = {}
        def allowSelfSignedHttps(allowed):
                """Bypass the server certificate verification on the client side."""
                if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
                    ssl._create_default_https_context = ssl._create_unverified_context

        allowSelfSignedHttps(True)  # This is needed if you use self-signed certificates.

        def send_chat_input_LLM(question):
            """Send a chat input to the API and return the parsed response."""
            # API request payload
            data = {
                "chat_input": question,
                "chat_history": []  # Modify this if you have previous chat history
            }
            
            # API credentials
            api_key = "OzYEqfLg8ueVePHKun3UejfBMLphos4N"  # Replace with your actual API key
            url = "https://ws-meemankraft-gcmko.eastus2.inference.ml.azure.com/score"  # Replace with the actual API URL

            # Prepare request body and headers
            body = str.encode(json.dumps(data))
            
            if not api_key:
                raise Exception("A valid API key is required to invoke the endpoint.")
            
            headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + api_key}
            req = urllib.request.Request(url, body, headers)
            
            # Initialize response structure
            text_docs = []
            final_dict = {}

            try:
                # Make the API call
                response = urllib.request.urlopen(req)
                result = response.read().decode('utf-8')
                
                # Parse the JSON response
                result_dict = json.loads(result)
                chat_output = result_dict.get('chat_output', {})
                
                # Extract chat response data
                response_output = chat_output.get('response', 'No response found')
                user_question = chat_output.get('user_question', 'No question found')
                source_documents = chat_output.get('source_documents', [])
                
                # Collect source document texts
                for doc in source_documents:
                    text = doc.get('text', 'No text found')
                    text_docs.append(text)
                
                # Build the response dictionary
                final_dict['response_output'] = response_output
                final_dict['user_question'] = user_question
                final_dict['source_documents'] = text_docs

                return final_dict

            except urllib.error.HTTPError as error:
                print(f"The request failed with status code: {error.code}")
                print(error.info())
                print(error.read().decode("utf8", 'ignore'))
                return None          
            
            return response_dict
        response_dictionary = send_chat_input_LLM(question)
        return response_dictionary

In [150]:
evaluator = RAG_Evaluator(framework = "deepeval")

In [178]:
# Apply the function to each row in the DataFrame
df['LLM_response'] = df.apply(lambda row: evaluator.get_rag_response(row['question']),axis=1)

In [182]:
# # If you want to separate the response_output and source_documents into different columns
df['response_output_LLM'] = df['LLM_response'].apply(lambda x: x['response_output'])
df['source_documents_LLM'] = df['LLM_response'].apply(lambda x: x['source_documents'])

In [170]:
# Apply the function to each row in the DataFrame
df['deepeval_response'] = df.apply(lambda row: evaluator.get_evaluation(row['question'], row['answer'], row['LLM_response']), axis=1)

✨ You're running DeepEval's latest Contextual Recall Metric! (using Custom Azure OpenAI Model, strict=False, asyn…

In [172]:

# If you want to separate the response_output and source_documents into different columns
df['answer_relevancy_score'] = df['deepeval_response'].apply(lambda x: x['answer_relevancy_score'])
df['contextual_relevancy_score'] = df['deepeval_response'].apply(lambda x: x['contextual_relevancy_score'])
df['contextual_precision_score'] = df['deepeval_response'].apply(lambda x: x['contextual_precision_score'])
df['contextual_recall_score'] = df['deepeval_response'].apply(lambda x: x['contextual_recall_score'])
# df = df.drop(columns=['deepeval_response'])

In [206]:
df['source_documents'][0]

KeyError: 'source_documents'

# New

In [ ]:
import os
import time
import urllib.request
import json
import ssl
from deepeval.metrics import AnswerRelevancyMetric, ContextualRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric
from deepeval.test_case import LLMTestCase
from deepeval.models.base_model import DeepEvalBaseLLM
from langchain_openai import AzureChatOpenAI

class AzureOpenAIEvaluator(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

In [192]:
import time
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score

nltk.download('punkt')

class RAG_Evaluator():
    def __init__(self, framework):
        self.framework = framework
        if self.framework == "deepeval":
            custom_model = AzureChatOpenAI(
                openai_api_version="2023-03-15-preview",
                azure_deployment="gpt4omini",
                azure_endpoint="https://aoi-meemank.openai.azure.com",
                openai_api_key="772624b8d13c4d19a12e77108401695e",
            )
            azure_openai = AzureOpenAIEvaluator(model=custom_model)
            self.answer_relevancy_metric = AnswerRelevancyMetric(
                model=azure_openai,
                threshold=0.7,
                include_reason=True
            )
            self.context_relevancy_metric = ContextualRelevancyMetric(
                model=azure_openai,
                threshold=0.7,
                include_reason=True
            )
            self.context_precision_metric = ContextualPrecisionMetric(
                threshold=0.7,
                model=azure_openai,
                include_reason=True
            )
            self.context_recall_metric = ContextualRecallMetric(
                threshold=0.7,
                model=azure_openai,
                include_reason=True
            )

    def check_answer_relevancy(self, question, actual_output):
        score = ""
        reason = ""

        if self.framework == "deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output
            )
            self.answer_relevancy_metric.measure(test_case)
            score = self.answer_relevancy_metric.score
            reason = self.answer_relevancy_metric.reason

        return score, reason

    def check_contextual_relevancy(self, question, actual_output, retrieval_contexts):
        score = ""
        reason = ""

        if self.framework == "deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output,
                retrieval_context=retrieval_contexts
            )
            self.context_relevancy_metric.measure(test_case)
            score = self.context_relevancy_metric.score
            reason = self.context_relevancy_metric.reason

        return score, reason

    def check_contextual_precision(self, question, actual_output, expected_output, retrieval_contexts):
        score = ""
        reason = ""

        if self.framework == "deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output,
                expected_output=expected_output,
                retrieval_context=retrieval_contexts
            )
            self.context_precision_metric.measure(test_case)
            score = self.context_precision_metric.score
            reason = self.context_precision_metric.reason

        return score, reason

    def check_contextual_recall(self, question, actual_output, expected_output, retrieval_contexts):
        score = ""
        reason = ""

        if self.framework == "deepeval":
            test_case = LLMTestCase(
                input=question,
                actual_output=actual_output,
                expected_output=expected_output,
                retrieval_context=retrieval_contexts
            )
            self.context_recall_metric.measure(test_case)
            score = self.context_recall_metric.score
            reason = self.context_recall_metric.reason

        return score, reason

    def calculate_metrics(self, ground_truth, llm_response):
        def calculate_rouge(reference, hypothesis):
            scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
            scores = scorer.score(reference, hypothesis)
            return scores

        def calculate_bertscore(reference, hypothesis):
            P, R, F1 = score([hypothesis], [reference], lang="en", verbose=False)
            return {
                'precision': P.item(),
                'recall': R.item(),
                'f1': F1.item()
            }

        rouge_scores = calculate_rouge(ground_truth, llm_response)
        bertscore = calculate_bertscore(ground_truth, llm_response)

        results = {
            'ROUGE-1': rouge_scores['rouge1'].fmeasure,
            'ROUGE-2': rouge_scores['rouge2'].fmeasure,
            'ROUGE-L': rouge_scores['rougeL'].fmeasure,
            'BERTScore Precision': bertscore['precision'],
            'BERTScore Recall': bertscore['recall'],
            'BERTScore F1': bertscore['f1']
        }

        return results

    def get_evaluation(self, question, expected_response, rag_response):
        rag_response_str = rag_response["response_output"]
        metrics = {}
        
        answer_relevancy_score, answer_relevancy_reason = self.check_answer_relevancy(question, rag_response_str)
        time.sleep(5)

        relevant_contexts = [source_doc for source_doc in rag_response["source_documents"]]
        contextual_relevancy_score, contextual_relevancy_reason = self.check_contextual_relevancy(question, rag_response_str, relevant_contexts)
        time.sleep(5)

        if expected_response != "":
            contextual_precision_score, contextual_precision_reason = self.check_contextual_precision(question, rag_response_str, expected_response, relevant_contexts)
            time.sleep(5)
            contextual_recall_score, contextual_recall_reason = self.check_contextual_recall(question, rag_response_str, expected_response, relevant_contexts)
            time.sleep(5)
        else:
            contextual_precision_score, contextual_precision_reason = 0.0, "NA"
            contextual_recall_score, contextual_recall_reason = 0.0, "NA"
        
        additional_metrics = self.calculate_metrics(expected_response, rag_response_str)

        metrics.update({
            'answer_relevancy_score': answer_relevancy_score,
            'answer_relevancy_reason': answer_relevancy_reason,
            'contextual_relevancy_score': contextual_relevancy_score,
            'contextual_relevancy_reason': contextual_relevancy_reason,
            'contextual_precision_score': contextual_precision_score,
            'contextual_precision_reason': contextual_precision_reason,
            'contextual_recall_score': contextual_recall_score,
            'contextual_recall_reason': contextual_recall_reason
        })

        metrics.update(additional_metrics)

        return metrics

    def get_rag_response(self, question):
        response = ""
        final_dict = {}
        def allowSelfSignedHttps(allowed):
                """Bypass the server certificate verification on the client side."""
                if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
                    ssl._create_default_https_context = ssl._create_unverified_context

        allowSelfSignedHttps(True)  # This is needed if you use self-signed certificates.

        def send_chat_input_LLM(question):
            """Send a chat input to the API and return the parsed response."""
            # API request payload
            data = {
                "chat_input": question,
                "chat_history": []  # Modify this if you have previous chat history
            }
            
            # API credentials
            api_key = "OzYEqfLg8ueVePHKun3UejfBMLphos4N"  # Replace with your actual API key
            url = "https://ws-meemankraft-gcmko.eastus2.inference.ml.azure.com/score"  # Replace with the actual API URL

            # Prepare request body and headers
            body = str.encode(json.dumps(data))
            
            if not api_key:
                raise Exception("A valid API key is required to invoke the endpoint.")
            
            headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + api_key}
            req = urllib.request.Request(url, body, headers)
            
            # Initialize response structure
            text_docs = []
            final_dict = {}

            try:
                # Make the API call
                response = urllib.request.urlopen(req)
                result = response.read().decode('utf-8')
                
                # Parse the JSON response
                result_dict = json.loads(result)
                chat_output = result_dict.get('chat_output', {})
                
                # Extract chat response data
                response_output = chat_output.get('response', 'No response found')
                user_question = chat_output.get('user_question', 'No question found')
                source_documents = chat_output.get('source_documents', [])
                
                # Collect source document texts
                for doc in source_documents:
                    text = doc.get('text', 'No text found')
                    text_docs.append(text)
                
                # Build the response dictionary
                final_dict['response_output'] = response_output
                final_dict['user_question'] = user_question
                final_dict['source_documents'] = text_docs

                return final_dict

            except urllib.error.HTTPError as error:
                print(f"The request failed with status code: {error.code}")
                print(error.info())
                print(error.read().decode("utf8", 'ignore'))
                return None          
            
            return response_dict
        response_dictionary = send_chat_input_LLM(question)
        return response_dictionary

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [202]:
# Read ground truth
import pandas as pd
import json

# Path to your JSONL file
file_path = "lab_maintenance_100_qa.jsonl"

# Read the JSON file as a list of dictionaries
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)  # Use json.load() since it's a JSON array, not a JSONL file

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df.head()

df=df.head(2)

In [204]:
evaluator = RAG_Evaluator(framework = "deepeval")
# Apply the function to each row in the DataFrame
df['LLM_response'] = df.apply(lambda row: evaluator.get_rag_response(row['question']),axis=1)
# # If you want to separate the response_output and source_documents into different columns
df['response_output_LLM'] = df['LLM_response'].apply(lambda x: x['response_output'])
df['source_documents_LLM'] = df['LLM_response'].apply(lambda x: x['source_documents'])
# Apply the function to each row in the DataFrame
df['deepeval_response'] = df.apply(lambda row: evaluator.get_evaluation(row['question'], row['answer'], row['LLM_response']), axis=1)

# If you want to separate the response_output and source_documents into different columns
df['answer_relevancy_score'] = df['deepeval_response'].apply(lambda x: x['answer_relevancy_score'])
df['contextual_relevancy_score'] = df['deepeval_response'].apply(lambda x: x['contextual_relevancy_score'])
df['contextual_precision_score'] = df['deepeval_response'].apply(lambda x: x['contextual_precision_score'])
df['contextual_recall_score'] = df['deepeval_response'].apply(lambda x: x['contextual_recall_score'])
# df = df.drop(columns=['deepeval_response'])

✨ You're running DeepEval's latest Contextual Recall Metric! (using Custom Azure OpenAI Model, strict=False, asyn…

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [205]:
df['ROUGE-1'] = df['deepeval_response'].apply(lambda x: x['ROUGE-1'])
df['ROUGE-2'] = df['deepeval_response'].apply(lambda x: x['ROUGE-2'])
df['ROUGE-L'] = df['deepeval_response'].apply(lambda x: x['ROUGE-L'])
df['BERTScore_Precision'] = df['deepeval_response'].apply(lambda x: x['BERTScore Precision'])
df['BERTScore_Recall'] = df['deepeval_response'].apply(lambda x: x['BERTScore Recall'])
df['BERTScore_F1'] = df['deepeval_response'].apply(lambda x: x['BERTScore F1'])
# df = df.drop(columns=['deepeval_response'])

In [208]:
df['source_documents_LLM'][0]

['Title: data.pdfMAINTENANCE MANUAL FOR LABORATORY EQUIPMENT\n1The microplate reader also known as “Photometric \nmicro-plate reader or ELISA reader” is a specialized spectrophotometer designed to read results of the ELISA test, a technique used to determine the presence of antibodies or specifi  c antigens in samples. The technique \nis based on the detection of an antigen or antibodies captured on a solid surface using direct or secondary, labelled antibodies, producing a reaction whose product can be read by the spectrophotometer. The word ELISA is the acronym for “Enzyme-Linked Immunosorbent Assay” . This chapter covers the use of microplate readers for ELISA testing. For additional information on the instrument principles of operation and maintenance, consult Chapter 11 discussing the spectrophotometer.\nPHOTOGRAPH OF MICROPLATE READER\nPURPOSE OF THE MICROPLATE READER\nThe microplate reader is used for reading the results of ELISA \ntests. This technique has a direct application 